In [3]:
import sys
import os

current_dir = os.getcwd() 
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

In [4]:
from Tbx_Integracao import *

import pandas as pd
import pickle

from unidecode import unidecode
from datetime import datetime

***
***

## API CEP -> endereço
Confirma somente que a rua está relacionada com o CEP (não checa número -> verificador do CEP)

In [6]:
df_cep_checked = pd.read_pickle("data_clean/cep_checked.pkl")

print('CEPs verificados: 50395')
print('CEPs confirmados:', df_cep_checked.shape[0])
df_cep_checked.head()

CEPs verificados: 50395
CEPs confirmados: 43625


,CEP,DESC_RUA,Complemento,Status
11194,08270155,TRAVESSA MARIA MAYER,NaN,True
5551,04864230,ANDORINHA CRISTAL,NaN,True
15108,04864200,ANUM-PRETO,NaN,True
10827,04864220,ARARA AZUL,NaN,True
13126,04864180,BEM-TE-VI AMARELO,NaN,True


***
***

# Arquivos:

## **SABESP**
file_path = 'data/COORDENADAS_BOS_JUNHO_2024.csv'

file_saved = 'data_clean/data_SP.pkl'

In [7]:
# @Carregar arquivo salvo
# data_SP_checked = pd.read_pickle("data_clean/data_SP.pkl")

In [20]:
# @Alternativamente: Importação arquivo com endereços e PDEs
data_SP = pd.read_csv('../data/coordenadas_SP.csv')

col_size = ["CEP", "DESC_RUA", "NUMERO_RUA"]

# print('Tamanho inicial (capital):\t\t', data_SP[col_size].drop_duplicates().shape[0])

# Correção CEP
data_SP['CEP'] = data_SP['CEP'].fillna(19999991).astype(int).astype(str)
data_SP['CEP'] = data_SP['CEP'].apply(lambda x: x.zfill(8))

# data_SP['CEP'] = data_SP['CEP'].apply(
#     lambda x: x.zfill(8)[:5] + '-' + x.zfill(8)[5:] if len(x) == 7 else x[:5] + '-' + x[5:]
# )

# Correção padrão PDE (10 dígitos)
data_SP['PDE'] = data_SP['PDE'].astype(str).apply(
    lambda x: x.zfill(10)
    )

#unica rua com essa formatação
data_SP['DESC_RUA'] = data_SP['DESC_RUA'].str.replace('25 DE', 'VINTE E CINCO DE')
data_SP = data_SP[(data_SP['CEP'] != '11111-111') & (~data_SP['COORD_Y'].isna())]

# Colunas irrelevantes
data_SP.drop(columns = ['Unnamed: 0', 'DATA_LEITURA', 'COMPLEMENTO'], inplace = True)

# Arredondamento Lat/Long -> tolerância ~15m por 0.001 de diferença
data_SP['LAT'] = data_SP['LAT'].astype(str).str.replace(',', '.').astype(float).round(4)
data_SP['LONG'] = data_SP['LONG'].astype(str).str.replace(',', '.').astype(float).round(4)

# Remover duplicados
data_SP = data_SP.drop_duplicates()

# Inclusão status de CEP -> se confirmado no scrapping
data_SP_checked = pd.merge(data_SP, df_cep_checked, how = 'left', on = ['DESC_RUA', 'CEP'])
data_SP_checked['Status'] = data_SP_checked['Status'].fillna(False)
data_SP_checked['CEP'] = data_SP_checked['CEP'].apply(
    lambda x: x[:5] + '-' + x[5:] if isinstance(x, str) and len(x) >= 6 else np.nan)

# Relatorio final
# print(f'{"Tamanho inicial:" :<40}', data_SP[col_size].drop_duplicates().shape[0])
print(f'\n{"Endereços (capital):" :<40}', data_SP[col_size].drop_duplicates().shape[0])
print(f'{"Endereços CEP confirmado:" :<40}',
      data_SP_checked[data_SP_checked['Status'] == True][col_size].drop_duplicates().shape[0])
print(f'\n{"PDEs únicos:" :<40}', data_SP_checked['PDE'].unique().shape[0])
print(f'\t{"Endereço com PDE único:" :<32}', sum(
    data_SP_checked[['DESC_RUA', 'NUMERO_RUA', 'CEP', 'PDE']].groupby(col_size)['PDE'].count() == 1))
print(f'\t{"Endereços com vários PDEs:" :<32}', len(
    data_SP_checked[['DESC_RUA', 'NUMERO_RUA', 'CEP', 'PDE']].groupby(col_size)['PDE'].count() > 1))
print(f'\t\t{"PDEs agrupados:" :<24}', (data_SP_checked['PDE'].unique().shape[0] - sum(
    data_SP_checked[['DESC_RUA', 'NUMERO_RUA', 'CEP', 'PDE']].groupby(col_size)['PDE'].count() == 1)))

data_SP_checked.to_pickle("data_clean/data_SP.pkl")

data_SP_checked.head()


Endereços (capital):                     1752689
Endereços CEP confirmado:                1367219

PDEs únicos:                             2611777
	Endereço com PDE único:          1313724
	Endereços com vários PDEs:       1752689
		PDEs agrupados:          1298053


,PDE,CODE_ENTIDADE,DESC_ENTIDADE,ATC,CEP,MUNICIPIO,TIPO_RUA,DESC_RUA,NUMERO_RUA,COORD_Y,LAT,LONG,Complemento,Status
0,2000478360,1027,OCSM,919,03385-100,SAO PAULO,RUA,LUIS MARIN DOS SANTOS,87.0,52511007.0,-23.5840,-46.5251,NaN,True
1,0746231849,1027,OCSM,919,03385-060,SAO PAULO,RUA,LAZARO GONCALVES FRAGA,58.0,5254766.0,-23.5829,-46.5255,NaN,True
2,0929391632,1027,OCSM,919,03385-050,SAO PAULO,RUA,SERAFIM DE ABREU,8.0,5258927.0,-23.5844,-46.5259,NaN,True
3,0125062575,1027,OCSM,919,03385-090,SAO PAULO,RUA,ACAPORI,12.0,5252262.0,-23.5833,-46.5252,NaN,False
4,0604648529,1027,OCSM,919,03729-100,SAO PAULO,RUA,FALANTO,68.0,52607766.0,-23.5826,-46.5261,NaN,False


***
***
***

## **GeoEmbraesp**

file_path = 'data/CargaDados.csv'

file_saved = 'data_clean/data_GEO_SP.pkl'

In [5]:
# @Carregar arquivo salvo
# data_GEO_SP = pd.read_pickle("data_clean/data_GEO_SP.pkl")

In [6]:
# @Dados correção monetária
IGPM = pd.read_csv('../data/FGV_IGPM.csv', delimiter = ';', encoding = 'latin1')
IGPM['Data'] = IGPM['Data'].apply(convert_to_datetime, format = '%m/%Y')

for col in IGPM.iloc[:, 1:]:
    IGPM[col] = IGPM[col].str.replace(',', '.').astype(float)

IGPM.drop(columns = 'IGP-M - Fechamento mensal (%)(1000367)', inplace = True)
IGPM.rename(columns = {'Data': 'data_lancamento',
                       'Índice Geral de Preços - IGP-M(200045)': 'IGP_M'}, inplace = True)

IGPM.head()

,data_lancamento,IGP_M
0,1989-06-01,0.000031
1,1989-07-01,0.000042
2,1989-08-01,0.000057
3,1989-09-01,0.000080
4,1989-10-01,0.000113


In [7]:
# @Padrão de colunas
id_cols = ['Empreendimento_Nome', 'DESC_RUA', 'NUMERO_RUA', 'CEP', 'Bairro', 'Ficha', 'LAT', 'LONG', 'Observacoes']

agg_cols = ['DESC_RUA', 'NUMERO_RUA', 'Imovel_residencial', 'Imovel_vertical', ]  #'data_lancamento' <- CONFIRMAR

predio_cols = ['data_lancamento', 'Idade_predio', 'Blocos', 'Unidades_andar', 'Andares_tipo', 'Total_Unidades',
               'Elevadores', 'Coberturas', 'Dormitorios', 'Banheiros_Sociais',
               'Vagas_Garagem', 'M2_Terreno', ]

unidade_cols = ['M2_util_unidade_tipo', 'M2_total_unidade_tipo',
                'M2_util_cobertura', 'M2_total_cobertura', ]

finance_cols = ['RS_total_unidade_tipo_IGPM', 'RS_por_M2_area_util_IGPM',
                'RS_por_M2_area_total_IGPM', 'RS_total_cobertura_IGPM', ]

numeric_cols = predio_cols + unidade_cols + finance_cols
full_cols = concatenate_unique([id_cols + agg_cols + numeric_cols])

In [8]:
# @Mapa nome de rua com datas
number_mapping = [
    ['PRIMEIRO DE', '1 DE'],
    ['SEGUNDO DE', '2 DE'],
    ['TRES DE', '3 DE'],
    ['QUATRO DE', '4 DE'],
    ['CINCO DE', '5 DE'],
    ['SEIS DE', '6 DE'],
    ['SETE DE', '7 DE'],
    ['OITO DE', '8 DE'],
    ['NOVE DE', '9 DE'],
    ['DEZ DE', '10 DE'],
    ['ONZE DE', '11 DE'],
    ['DOZE DE', '12 DE'],
    ['TREZE DE', '13 DE'],
    ['CATORZE DE', '14 DE'],
    ['QUINZE DE', '15 DE'],
    ['DEZESSEIS DE', '16 DE'],
    ['DEZESSETE DE', '17 DE'],
    ['DEZOITO DE', '18 DE'],
    ['DEZENOVE DE', '19 DE'],
    ['VINTE DE', '20 DE'],
    ['VINTE E UM DE', '21 DE'],
    ['VINTE E DOIS DE', '22 DE'],
    ['VINTE E TRÊS DE', '23 DE'],
    ['VINTE E QUATRO DE', '24 DE'],
    ['VINTE E CINCO DE', '25 DE'],
    ['VINTE E SEIS DE', '26 DE'],
    ['VINTE E SETE DE', '27 DE'],
    ['VINTE E OITO DE', '28 DE'],
    ['VINTE E NOVE DE', '29 DE'],
    ['TRINTA DE', '30 DE'],
    ['TRINTA E UM DE', '31 DE'], ]

In [9]:
# @Importação de dados imobiliários
data_GEO = pd.read_csv('../data/CargaDados_ahr.csv', delimiter = ';', encoding = 'latin1', dtype = str)
data_GEO['Empreendimento_Nome'] = data_GEO['NomeEmpreendimento'].combine_first(data_GEO['NomeEmpreendimentoF'])

# Filtragem capital
data_GEO['Cidade'] = data_GEO['Cidade'].str.upper().apply(unidecode)
data_GEO_SP = data_GEO[data_GEO['Cidade'] == 'SAO PAULO'].copy()

# Padronização endereço
data_GEO_SP[['TIPO_RUA', 'DESC_RUA']] = data_GEO_SP['Logradouro'].str.split(' ', n = 1, expand = True)
data_GEO_SP = data_GEO_SP[~data_GEO_SP['DESC_RUA'].isna()]
data_GEO_SP['DESC_RUA'] = data_GEO_SP['DESC_RUA'].str.upper().apply(unidecode)

for day in number_mapping[::-1]:  # Correção nomes de ruas com datas
    data_GEO_SP['DESC_RUA'] = data_GEO_SP['DESC_RUA'].str.strip().str.replace(str(day[1]), str(day[0])).copy()

data_GEO_SP.rename(columns = {'Numero': 'NUMERO_RUA'}, inplace = True)
data_GEO_SP['NUMERO_RUA'] = data_GEO_SP['NUMERO_RUA'].fillna('9999999')
data_GEO_SP = data_GEO_SP[~data_GEO_SP['NUMERO_RUA'].str.contains(r'[^0-9]')]
data_GEO_SP['NUMERO_RUA'] = data_GEO_SP['NUMERO_RUA'].astype(float)

# Padronização CEP
data_GEO_SP['CEP'] = data_GEO_SP['CEP'].fillna('-1')
data_GEO_SP['CEP'] = data_GEO_SP['CEP'].str.replace('-', '')
data_GEO_SP['CEP'] = data_GEO_SP['CEP'].apply(
    lambda x: x[:5] + '-' + x[5:] if isinstance(x, str) and len(x) >= 6 else np.nan)

# Arredondamento Lat/Long -> tolerância ~15m por 0.001 de diferença
data_GEO_SP['LAT'] = data_GEO_SP['Latitude'].str.replace(',', '.').astype(float).round(4)
data_GEO_SP['LONG'] = data_GEO_SP['Longitude'].str.replace(',', '.').astype(float).round(4)

# Padronização categorias Residencial/Comercial & Horizontal/Vertical 
data_GEO_SP['Imovel_residencial'] = np.where(data_GEO_SP['Tipo_imovel'].str.lower().str.contains('r'), 1, 0)
data_GEO_SP['Imovel_vertical'] = np.where(data_GEO_SP['Tipo_empreendimento'].str.lower().str.contains('v'), 1, 0)

# Correção tipo variáveis numéricas (float por conta de agregação)
data_GEO_SP['Andares_tipo'] = data_GEO_SP['Andares_tipo'].astype(float)
data_GEO_SP['Total_Unidades'] = data_GEO_SP['Total_Unidades'].astype(float)

# Padronização data de lançamento e criação variável idade
data_GEO_SP = data_GEO_SP[
    ~data_GEO_SP['Lancamento'].isna()]  # ATENÇÃO, FEITO SÓ PORQUE TEM 2 CASOS, CHECAR CASO NECESSÁRIO
data_GEO_SP['data_lancamento'] = data_GEO_SP['Lancamento'].apply(convert_to_datetime)
today = pd.Timestamp(datetime.today())
data_GEO_SP['Idade_predio'] = ((today - data_GEO_SP['data_lancamento']).dt.days / 365.25).astype(int)

# Aplicando correção monetária
data_GEO_SP = pd.merge(data_GEO_SP, IGPM, how = 'left', on = 'data_lancamento')
data_GEO_SP['Fator_correcao_IGPM'] = IGPM.iloc[-1]['IGP_M'] / data_GEO_SP['IGP_M']

# Correção valores monetarios  
for col in [col for col in data_GEO_SP.columns if ('M2' in col) or ('RS' in col)]:
    data_GEO_SP[col] = data_GEO_SP[col].str.replace('---', '0').astype(float)
    if 'RS' in col:
        data_GEO_SP[col + '_IGPM'] = data_GEO_SP[col] * data_GEO_SP['Fator_correcao_IGPM']
        data_GEO_SP[col + '_IGPM'] = data_GEO_SP[col + '_IGPM'].fillna(-1)
        data_GEO_SP.drop(columns = col, inplace = True)

# COrreção condominios horizontais
data_GEO_SP.loc[data_GEO_SP['Blocos'].isna(), 'Elevadores'] = 0
data_GEO_SP.loc[data_GEO_SP['Blocos'].isna(), 'Unidades_andar'] = data_GEO_SP.loc[
    data_GEO_SP['Blocos'].isna(), 'Total_Unidades']
data_GEO_SP.loc[data_GEO_SP['Blocos'].isna(), 'Coberturas'] = 0
data_GEO_SP.loc[data_GEO_SP['Blocos'].isna(), 'Blocos'] = 1

data_GEO_SP = data_GEO_SP.drop_duplicates()

print(
    f'{"Tamanho inicial:" :<30}{data_GEO[["CEP", "Logradouro", "Numero"]].drop_duplicates().shape[0]:<6} (Endereços únicos)')
print(f'{"Tamanho final (capital):" :<30}{data_GEO_SP[col_size].drop_duplicates().shape[0]:<6} (Endereços únicos)')

data_GEO_SP = data_GEO_SP[full_cols]

# Correção tipo
for col in data_GEO_SP.columns[13:]:
    data_GEO_SP[col] = data_GEO_SP[col].astype(float)

data_GEO_SP.to_pickle("data_clean/data_GEO_SP.pkl")

data_GEO_SP.head()

Tamanho inicial:              15481  (Endereços únicos)
Tamanho final (capital):      12319  (Endereços únicos)


,Empreendimento_Nome,DESC_RUA,NUMERO_RUA,CEP,Bairro,Ficha,LAT,LONG,Observacoes,Imovel_residencial,...,Vagas_Garagem,M2_Terreno,M2_util_unidade_tipo,M2_total_unidade_tipo,M2_util_cobertura,M2_total_cobertura,RS_total_unidade_tipo_IGPM,RS_por_M2_area_util_IGPM,RS_por_M2_area_total_IGPM,RS_total_cobertura_IGPM
0,MERIDIAN BARRA FUNDA,DR. RUBENS MEIRELES,374.0,NaN,VÁRZEA DA BARRA FUNDA,FICHA A,-23.5181,-46.6655,"UNIDADES-TIPO DIFERENCIADAS, VIDE FICHAS B E C...",1,...,2.0,6035.62,93.93,164.0,NaN,NaN,1.918693e+06,20426.831447,11699.356961,-1.0
1,MERIDIAN BARRA FUNDA,DR. RUBENS MEIRELES,374.0,NaN,VÁRZEA DA BARRA FUNDA,FICHA B,-23.5181,-46.6655,"UNIDADES-TIPO DIFERENCIADAS, VIDE FICHAS A E C...",1,...,2.0,663.42,112.69,197.0,NaN,NaN,2.301945e+06,20427.244854,11685.009322,-1.0
2,MERIDIAN BARRA FUNDA,DR. RUBENS MEIRELES,374.0,NaN,VÁRZEA DA BARRA FUNDA,FICHA C,-23.5181,-46.6655,"UNIDADES-TIPO DIFERENCIADAS, VIDE FICHAS A E B...",1,...,3.0,5371.71,112.69,216.0,NaN,NaN,2.405053e+06,21342.210839,11134.497694,-1.0
3,RESIDENCIAL DAS FLORES,JOSE DE CAMPOS NOVAIS,115.0,02834-020,VILA NINA,FICHA A,-23.4732,-46.6915,TRATA-SE DE CONDOMÍNIO HORIZONTAL.,1,...,2.0,1277.00,63.00,103.0,NaN,NaN,6.225416e+05,9881.608325,6044.101153,-1.0
4,RESIDENCIAL COTTI,COMENDADOR GABRIEL COTTI,113.0,02919-110,VILA PEREIRA BARRETO,FICHA A,-23.4917,-46.7164,UNID/AND: 4 UNID NO TÉRREO E 5 DO 1º AO 8º AND...,1,...,1.0,553.54,52.00,85.0,NaN,NaN,6.658452e+05,12804.708825,7833.470645,-1.0


***
***
***

## IPTU (nova base)

file_saved = 'data_clean/IPTU_full.pkl'

file_saved = 'data_clean/IPTU_agg.pkl'

In [10]:
# @Carregar arquivo salvo
# iptu24 = pd.read_pickle("data_clean/IPTU_full.pkl")
# iptu_agg = pd.read_pickle("data_clean/IPTU_agg.pkl")

In [11]:
# @Importação de dados IPTU
iptu24 = pd.read_csv('../data/IPTU/IPTU_2024.csv', delimiter = ';', encoding = 'latin1', dtype = str)

# FIltragem imóveis sem número
iptu24 = iptu24[~iptu24['NUMERO DO IMOVEL'].isna()].copy()
iptu24['NUMERO DO IMOVEL'] = iptu24['NUMERO DO IMOVEL'].astype(int)

# Filtragem colunas relevantes
iptu24 = iptu24[['NUMERO DO CONTRIBUINTE', 'TIPO DE USO DO IMOVEL', 'NOME DE LOGRADOURO DO IMOVEL', 'NUMERO DO IMOVEL',
                 'BAIRRO DO IMOVEL',
                 'CEP DO IMOVEL', 'QUANTIDADE DE PAVIMENTOS', 'QUANTIDADE DE ESQUINAS/FRENTES', 'FRACAO IDEAL',
                 'AREA DO TERRENO', 'AREA CONSTRUIDA', 'AREA OCUPADA',
                 'VALOR DO M2 DO TERRENO', 'VALOR DO M2 DE CONSTRUCAO',
                 'TESTADA PARA CALCULO',
                 'TIPO DE PADRAO DA CONSTRUCAO', 'TIPO DE TERRENO',
                 'FATOR DE OBSOLESCENCIA']]

iptu24 = iptu24.rename(columns = {'NOME DE LOGRADOURO DO IMOVEL': 'DESC_RUA',
                                  'NUMERO DO IMOVEL': 'NUMERO_RUA',
                                  'CEP DO IMOVEL': 'CEP'
                                  })

iptu24.head()

,NUMERO DO CONTRIBUINTE,TIPO DE USO DO IMOVEL,DESC_RUA,NUMERO_RUA,BAIRRO DO IMOVEL,CEP,QUANTIDADE DE PAVIMENTOS,QUANTIDADE DE ESQUINAS/FRENTES,FRACAO IDEAL,AREA DO TERRENO,AREA CONSTRUIDA,AREA OCUPADA,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,TESTADA PARA CALCULO,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA
0,0010030001-4,Loja,R S CAETANO,13,SANTA EFIGENIA,01104-001,01,1,1.0000,0000136,0000135,0000108,0003674.00,0002711.00,00013.00,Comercial horizontal - padrão B,De esquina,0.20
1,0010030002-2,Loja,R S CAETANO,19,SANTA EFIGENIA,01104-001,01,0,1.0000,0000090,0000067,0000067,0003674.00,0002711.00,00006.00,Comercial horizontal - padrão B,Normal,0.20
2,0010030003-0,Loja,R S CAETANO,27,SANTA EFIGENIA,01104-001,02,0,1.0000,0000105,0000140,0000084,0003674.00,0002711.00,00007.85,Comercial horizontal - padrão B,Normal,0.22
3,0010030004-9,Loja,R S CAETANO,33,NaN,01104-001,01,0,1.0000,0000108,0000103,0000086,0003674.00,0002711.00,00006.05,Comercial horizontal - padrão B,Normal,0.20
4,0010030005-7,Loja,R S CAETANO,39,NaN,01104-001,02,0,1.0000,0000120,0000170,0000110,0003674.00,0002711.00,00006.70,Comercial horizontal - padrão B,Normal,0.82


In [ ]:
# Valores possíveis para tipo de uso -> filtragem para contagem de unidade somente com usos residenciais
['Apartamento em condomínio',
 'Flat residencial em condomínio',
 'Prédio de apartamento, não em condomínio, de uso exclusivamente residencial',
 'Prédio de apartamento, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios), com ou sem loja (predominância residencial)',
 'Residencial horizontal - padrão B',
 'Residência',
 'Residência coletiva, exclusive cortiço (mais de uma residência no lote)',
 'Residência e outro uso (predominância residencial)',
 'Cortiço',
 'Depósito Condomínio Residencial',

 'Armazéns gerais e depósitos',
 'Asilo, orfanato, creche, seminário ou convento',
 'Cinema, teatro, casa de diversão, clube ou congênere',
 'Clube esportivo',

 'Escola',
 'Escritório/consultório em condomínio (unidade autônoma)',
 'Estação radioemissora, de televisão ou empresa jornalística',

 'Flat de uso comercial (semelhante a hotel)',
 'Prédio de escritório ou consultório, não em condomínio, com ou sem lojas',
 'Prédio de escritório, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios) com ou sem loja (predominância comercial)',

 'Garagem (exclusive em prédio em condomínio)',
 'Garagem (unidade autônoma) de prédio de garagens',
 'Garagem (unidade autônoma) em edifício em condomínio de escritórios, consultórios ou misto',
 'Garagem (unidade autônoma) em edifício em condomínio de uso exclusivamente residencial',

 'Hospital, ambulatório, casa de saúde e assemelhados',
 'Hotel, pensão ou hospedaria',
 'Indústria',
 'Loja',
 'Loja e residência (predominância comercial)',
 'Loja em edifício em condomínio (unidade autônoma)',

 'Normal',  #?????????????????

 'Não residencial',
 'Oficina',
 'Outras edificações de uso coletivo, com utilização múltipla',
 'Outras edificações de uso comercial, com utilização múltipla',
 'Outras edificações de uso de serviço, com utilização múltipla',
 'Outras edificações de uso especial, com utilização múltipla',
 'Posto de serviço',

 'Templo',
 'Terreno']

In [12]:
aux_contagem_residencial = ['Apartamento em condomínio',
                            'Flat residencial em condomínio',
                            'Prédio de apartamento, não em condomínio, de uso exclusivamente residencial',
                            'Prédio de apartamento, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios), com ou sem loja (predominância residencial)',
                            'Residencial horizontal - padrão B',
                            'Residência',
                            'Residência coletiva, exclusive cortiço (mais de uma residência no lote)',
                            'Residência e outro uso (predominância residencial)',
                            'Cortiço',
                            'Depósito Condomínio Residencial', ]

iptu_residencial = iptu24[iptu24['TIPO DE USO DO IMOVEL'].isin(aux_contagem_residencial)]

In [13]:
# Criação de variável de contagem de imóveis por endereço
contagem_IPTU = iptu_residencial[
    ['CEP', 'NUMERO_RUA', 'TIPO DE PADRAO DA CONSTRUCAO', 'QUANTIDADE DE PAVIMENTOS',
     'NUMERO DO CONTRIBUINTE']].groupby(
    ['CEP', 'NUMERO_RUA', 'TIPO DE PADRAO DA CONSTRUCAO', 'QUANTIDADE DE PAVIMENTOS']).count().reset_index().rename(
    columns = {'NUMERO DO CONTRIBUINTE': 'CONTAGEM_UNIDADES'}).drop_duplicates()

iptu24 = pd.merge(iptu24, contagem_IPTU,
                  on = ['CEP', 'NUMERO_RUA', 'TIPO DE PADRAO DA CONSTRUCAO', 'QUANTIDADE DE PAVIMENTOS'], how = 'left').fillna(0)

print(
    f'{"Tamanho base:" :<30}{iptu24[["DESC_RUA", "NUMERO_RUA", "CEP"]].drop_duplicates().shape[0]:<10} (Endereços únicos)')

iptu24.to_pickle("data_clean/IPTU_full.pkl")

iptu24.head()

Tamanho base:                 1538829    (Endereços únicos)


,NUMERO DO CONTRIBUINTE,TIPO DE USO DO IMOVEL,DESC_RUA,NUMERO_RUA,BAIRRO DO IMOVEL,CEP,QUANTIDADE DE PAVIMENTOS,QUANTIDADE DE ESQUINAS/FRENTES,FRACAO IDEAL,AREA DO TERRENO,AREA CONSTRUIDA,AREA OCUPADA,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,TESTADA PARA CALCULO,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,CONTAGEM_UNIDADES
0,0010030001-4,Loja,R S CAETANO,13,SANTA EFIGENIA,01104-001,01,1,1.0000,0000136,0000135,0000108,0003674.00,0002711.00,00013.00,Comercial horizontal - padrão B,De esquina,0.20,0.0
1,0010030002-2,Loja,R S CAETANO,19,SANTA EFIGENIA,01104-001,01,0,1.0000,0000090,0000067,0000067,0003674.00,0002711.00,00006.00,Comercial horizontal - padrão B,Normal,0.20,0.0
2,0010030003-0,Loja,R S CAETANO,27,SANTA EFIGENIA,01104-001,02,0,1.0000,0000105,0000140,0000084,0003674.00,0002711.00,00007.85,Comercial horizontal - padrão B,Normal,0.22,0.0
3,0010030004-9,Loja,R S CAETANO,33,0,01104-001,01,0,1.0000,0000108,0000103,0000086,0003674.00,0002711.00,00006.05,Comercial horizontal - padrão B,Normal,0.20,0.0
4,0010030005-7,Loja,R S CAETANO,39,0,01104-001,02,0,1.0000,0000120,0000170,0000110,0003674.00,0002711.00,00006.70,Comercial horizontal - padrão B,Normal,0.82,0.0


In [14]:
aux_iptu = iptu24[['DESC_RUA', 'NUMERO_RUA', 'CEP',
                   'TIPO DE PADRAO DA CONSTRUCAO',
                   'QUANTIDADE DE PAVIMENTOS', 'TESTADA PARA CALCULO',
                   'FATOR DE OBSOLESCENCIA', 'CONTAGEM_UNIDADES',
                   'VALOR DO M2 DO TERRENO', 'AREA DO TERRENO',
                   'VALOR DO M2 DE CONSTRUCAO', 'AREA CONSTRUIDA',
                   ]].copy()  #'TIPO DE USO DO IMOVEL',

aux_iptu['QUANTIDADE DE PAVIMENTOS'] = aux_iptu['QUANTIDADE DE PAVIMENTOS'].replace(to_replace = r'[a-zA-Z]+',
                                                                                    value = 1, regex = True)
aux_iptu['TESTADA PARA CALCULO'] = aux_iptu['TESTADA PARA CALCULO'].replace(to_replace = r'[a-zA-Z]+', value = 0,
                                                                            regex = True)

for col in aux_iptu.columns[4:]:
    aux_iptu[col] = aux_iptu[col].astype(float)

agg_functions = {'QUANTIDADE DE PAVIMENTOS': 'mean',
                 'TESTADA PARA CALCULO': 'mean',
                 'FATOR DE OBSOLESCENCIA': 'mean',
                 'CONTAGEM_UNIDADES': 'mean',
                 'VALOR DO M2 DO TERRENO': 'mean',
                 'AREA DO TERRENO': 'mean',
                 'VALOR DO M2 DE CONSTRUCAO': 'mean',
                 'AREA CONSTRUIDA': 'sum'}

iptu_agg = aux_iptu.groupby(aux_iptu.columns[:4].tolist()).agg(agg_functions).sort_values(
    ['CEP', 'NUMERO_RUA']).reset_index()

iptu_agg = iptu_agg.drop(columns = ['DESC_RUA'])

iptu_agg.to_pickle("data_clean/IPTU_agg.pkl")

iptu_agg.head()

,NUMERO_RUA,CEP,TIPO DE PADRAO DA CONSTRUCAO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,FATOR DE OBSOLESCENCIA,CONTAGEM_UNIDADES,VALOR DO M2 DO TERRENO,AREA DO TERRENO,VALOR DO M2 DE CONSTRUCAO,AREA CONSTRUIDA
0,0,-,TERRENO,0.0,13.77,0.00,0.0,663.5,327.75,0.0,0.0
1,0,-,TERRENO,0.0,52.00,0.00,0.0,433.0,902.00,0.0,0.0
2,0,-,Residencial horizontal - padrão A,1.0,11.00,0.29,2.0,1019.0,371.00,1676.0,35.0
3,0,-,TERRENO,0.0,11.00,0.00,0.0,1019.0,371.00,0.0,0.0
4,0,-,TERRENO,0.0,10.00,0.00,0.0,13674.0,279.00,0.0,0.0


***
***
***
***

# Integração das bases

## Caso restritivo 
file_saved = 'data_clean/iptu_geo_sabesp.pkl'

In [15]:
# @Carregar arquivo salvo
# iptu_geo_final = pd.read_pickle("data_clean/iptu_geo_sabesp.pkl")


### **Sem** tolerância (SABESP x GeoEmbraesp)

Dados agregados e agrupados
- file_saved = 'data_clean/GEO_endereco.pkl'

In [16]:
# @Carregar arquivo salvo
# df_GEO_endereco = pd.read_pickle("data_clean/GEO_endereco.pkl")

In [17]:
# @Merge dados SABESP x GeoEmbraesp

# cols_GEO = ['Empreendimento_Nome',
#            'Tipo_imovel', 'Tipo_empreendimento', 'Ficha', 'Blocos',
#            'Unidades_andar', 'Andares_tipo', 'Total_Unidades', 'Elevadores',
#            'Coberturas', 'Dormitorios', 'Banheiros_Sociais', 'Vagas_Garagem',
#            'M2_util_unidade_tipo', 'M2_total_unidade_tipo', 'M2_Terreno',
#            'M2_util_cobertura', 'M2_total_cobertura', 'RS_total_unidade_tipo',
#            'US_total_unidade_tipo', 'RS_por_M2_area_util', 'US_por_M2_area_util',
#            'RS_por_M2_area_total', 'US_por_M2_area_total', 'RS_total_cobertura',
#            'Lancamento', 'Logradouro', 'NUMERO_RUA', 'Bairro',
#            'Cidade', 'CEP', 'Observacoes', 'TIPO_RUA', 'DESC_RUA', 'LAT', 'LONG',
#            'Imovel_residencial', 'Imovel_vertical', 'Idade_predio']

cols_merge = ['PDE', 'CODE_ENTIDADE', 'CEP', 'TIPO_RUA', 'DESC_RUA', 'NUMERO_RUA', 'LAT', 'LONG', 'Status']

# Considerando correspondências de latitude e longitude
merge_GEO_LATLONG = pd.merge(data_GEO_SP,  #[cols_GEO]
                             data_SP_checked[cols_merge],
                             how = 'inner',
                             on = ['LAT', 'LONG'],
                             suffixes = ('', '_LATLONG'))

# Considerando correspondências de CEP e número da rua
merge_GEO_LOGRADOURO = pd.merge(data_GEO_SP,  #[cols_GEO]
                                data_SP_checked[cols_merge],
                                how = 'inner',
                                on = ['CEP', 'NUMERO_RUA'],
                                suffixes = ('', '_CEP'))

# Considerando correspondências de nome e número da rua
merge_GEO_RUA_NUM = pd.merge(data_GEO_SP,  #[cols_GEO]
                             data_SP_checked[cols_merge],
                             how = 'inner',
                             on = ['DESC_RUA', 'NUMERO_RUA'],
                             suffixes = ('', '_RUA'))

# Dataset final
df_GEO_merged = pd.concat([merge_GEO_LATLONG, merge_GEO_RUA_NUM, merge_GEO_LOGRADOURO], axis = 0).drop_duplicates()

# Correção de CEPs conflitantes
df_GEO_merged_cep = correct_cep(df_GEO_merged, apply_standard = False)

# df_GEO_merged_cep = df_GEO_merged_cep.drop(columns = ['CODE_ENTIDADE', 'CEP_LATLONG',
#                                                     'TIPO_RUA', 'DESC_RUA_LATLONG', 'NUMERO_RUA_LATLONG',
#                                                     'CEP_RUA', 'LAT_RUA', 'LONG_RUA', 'DESC_RUA_CEP', 'LAT_CEP',
#                                                     'LONG_CEP',
#                                                     'CEP_cod_lat', 'CEP_cod_rua', 'CEP_me_salve', 'CEP_cod', ]).drop_duplicates()

In [18]:
# @Relatório final integração - SEM TOLERÂNCIA
cols_endereco = ['Empreendimento_Nome', 'CEP', 'NUMERO_RUA', 'Imovel_residencial', 'Imovel_vertical', 'Status']
cols_pde = ['PDE']

print(f'Merge dados "coordenadas" x "carga dados" -> ligar PDE com endereço')

print(
    f'{"Size merge LATLONG:" :<30}{merge_GEO_LATLONG[cols_pde].drop_duplicates().shape[0]:<6} PDEs\n{"":<30}{merge_GEO_LATLONG[col_size].drop_duplicates().shape[0]:<6} Endereços únicos')

print(
    f'{"Size merge CEP_NUMERO:" :<30}{merge_GEO_LOGRADOURO[cols_pde].drop_duplicates().shape[0]:<6} PDEs\n{"":<30}{merge_GEO_LOGRADOURO[col_size].drop_duplicates().shape[0]:<6} Endereços únicos')

print(
    f'{"Size merge RUA_NUMERO:" :<30}{merge_GEO_RUA_NUM[cols_pde].drop_duplicates().shape[0]:<6} PDEs\n{"":<30}{merge_GEO_RUA_NUM[col_size].drop_duplicates().shape[0]:<6} Endereços únicos')

print(
    f'\n{"Size merge FINAL:" :<30}{df_GEO_merged_cep[cols_pde].drop_duplicates().shape[0]:<6} PDEs\n{"":<30}{df_GEO_merged_cep[col_size].drop_duplicates().shape[0]:<6} Endereços únicos')

df_GEO_merged_cep.head()

Merge dados "coordenadas" x "carga dados" -> ligar PDE com endereço
Size merge LATLONG:           5975   PDEs
                              2387   Endereços únicos
Size merge CEP_NUMERO:        13567  PDEs
                              3510   Endereços únicos
Size merge RUA_NUMERO:        26128  PDEs
                              5639   Endereços únicos

Size merge FINAL:             31721  PDEs
                              10837  Endereços únicos


,Empreendimento_Nome,DESC_RUA,NUMERO_RUA,Bairro,Ficha,LAT,LONG,Observacoes,Imovel_residencial,Imovel_vertical,...,LAT_RUA,LONG_RUA,DESC_RUA_CEP,LAT_CEP,LONG_CEP,CEP_cod_lat,CEP_cod_rua,CEP_me_salve,CEP_cod,CEP
0,RESIDENCIAL DAS FLORES,JOSE DE CAMPOS NOVAIS,115.0,VILA NINA,FICHA A,-23.4732,-46.6915,TRATA-SE DE CONDOMÍNIO HORIZONTAL.,1,0,...,NaN,NaN,NaN,NaN,NaN,02834-020,NaN,02834-020,NaN,02834-020
1,GREEN SOLARIUM RESIDENCE,RIO VERDE,280.0,VILA PALMEIRAS,FICHA A,-23.4848,-46.7014,"UNIDADES-TIPO DIFERENCIADAS, VIDE FICHA B. BLO...",1,1,...,NaN,NaN,NaN,NaN,NaN,02934-001,NaN,02934-001,NaN,02934-001
2,GREEN SOLARIUM RESIDENCE,RIO VERDE,280.0,VILA PALMEIRAS,FICHA B,-23.4848,-46.7014,"UNIDADES-TIPO DIFERENCIADAS, VIDE FICHA A. BLO...",1,1,...,NaN,NaN,NaN,NaN,NaN,02934-001,NaN,02934-001,NaN,02934-001
3,CONDOMÍNIO CAMPO DAS PITANGUEIRAS,CAMPO DAS PITANGUEIRAS,512.0,JARDIM SÃO NICOLAU,FICHA A,-23.5231,-46.4779,TRATA-SE DE CONDOMÍNIO HORIZONTAL.,1,0,...,NaN,NaN,NaN,NaN,NaN,03685-080,NaN,03685-080,NaN,03685-080
4,INSPIRE JARDIM PENHA,CORONEL RODOLFO PORTO,9999999.0,VILA MONTE SANTO,FICHA A,-23.5066,-46.4738,UNID/AND: 4 UNID NO TÉRREO E 6 DO 1º AO 8º ANDAR.,1,1,...,NaN,NaN,NaN,NaN,NaN,08062-090,NaN,08062-090,NaN,08062-090


In [19]:
# @Agregação dos dados para conferência com IPTU

agg_cols = ['Empreendimento_Nome', 'DESC_RUA', 'NUMERO_RUA', 'Imovel_residencial', 'Imovel_vertical',
            'CEP']

agg_functions = {
    'Idade_predio': 'mean',
    'Blocos': 'sum',
    'Unidades_andar': 'mean',
    'Andares_tipo': 'mean',
    'Total_Unidades': 'sum',
    'Elevadores': 'sum',
    'Coberturas': 'sum',
    'Dormitorios': 'mean',
    'Banheiros_Sociais': 'mean',
    'Vagas_Garagem': 'mean',
    'M2_util_unidade_tipo': 'mean',
    'M2_total_unidade_tipo': 'mean',
    'M2_Terreno': 'mean',
    'M2_util_cobertura': 'mean',
    'M2_total_cobertura': 'mean',
    'RS_total_unidade_tipo_IGPM': 'mean',
    'RS_por_M2_area_util_IGPM': 'mean',
    'RS_por_M2_area_total_IGPM': 'mean',
    'RS_total_cobertura_IGPM': 'mean',

    # 'data_lancamento' : 'mean',
    }

# Colunas de interesse
df_GEO_endereco = df_GEO_merged_cep[agg_cols + list(agg_functions.keys())].drop_duplicates().copy()

df_GEO_endereco = df_GEO_endereco.groupby(agg_cols).agg(agg_functions).sort_values(
    ['DESC_RUA', 'NUMERO_RUA', ]).reset_index()
print(f'Tamanho final da base de consumidores: {df_GEO_endereco[col_size].drop_duplicates().shape[0]}')

df_GEO_endereco.to_pickle("data_clean/GEO_endereco.pkl")

df_GEO_endereco.head()

Tamanho final da base de consumidores: 7327


,Empreendimento_Nome,DESC_RUA,NUMERO_RUA,Imovel_residencial,Imovel_vertical,CEP,Idade_predio,Blocos,Unidades_andar,Andares_tipo,...,Vagas_Garagem,M2_util_unidade_tipo,M2_total_unidade_tipo,M2_Terreno,M2_util_cobertura,M2_total_cobertura,RS_total_unidade_tipo_IGPM,RS_por_M2_area_util_IGPM,RS_por_M2_area_total_IGPM,RS_total_cobertura_IGPM
0,NAO DEFINIDO,ABACATUAJA,500.0,1,0,08190-420,11.0,1.0,8.0,2.0,...,1.0,48.00,60.00,500.00,NaN,NaN,354125.270973,7377.602434,5902.095227,-1.000000e+00
1,PARQUE DOS DOURADOS,ABADIA DOS DOURADOS,166.0,1,1,05000-000,29.0,1.0,4.0,8.0,...,2.0,67.15,157.79,1127.31,113.0,223.0,767993.343318,11436.936658,4867.157813,1.222726e+06
2,GOLDEN PALACE,ABAGIBA,540.0,1,1,04294-000,29.0,1.0,4.0,8.0,...,1.0,58.66,106.59,1000.00,NaN,NaN,590835.653240,10073.069880,5543.200727,-1.000000e+00
3,RESERVA SAÚDE,ABAGIBA,582.0,1,1,04294-000,8.0,1.0,6.0,17.0,...,1.5,63.48,124.71,1467.50,NaN,NaN,789487.261442,12436.784078,6362.585522,-1.000000e+00
4,MISSOURI,ABAGIBA,583.0,1,1,04294-000,29.0,1.0,4.0,19.0,...,1.0,74.14,126.71,1492.91,NaN,NaN,835931.794958,11275.068377,6597.204216,-1.000000e+00



### **Com** tolerância *parcial* (MERGE x IPTU)

Dados agregados e agrupados
- file_saved = 'data_clean/iptu_geo_sabesp.pkl'

In [20]:
## Integração CEPS x IPTU
iptu_geo = pd.merge(df_GEO_endereco,
                    iptu_agg,
                    #[['CEP', 'NUMERO_RUA', 'QUANTIDADE DE PAVIMENTOS', 'FATOR DE OBSOLESCENCIA','TIPO DE PADRAO DA CONSTRUCAO', 'CONTAGEM_UNIDADES']],
                    on = ['CEP', 'NUMERO_RUA'], how = 'left').drop_duplicates()

iptu_geo_coded = convert_padrao_iptu(iptu_geo.copy(), drop_original_col = True,
                                     compute_obsolence = True).drop_duplicates()

print(
    f"\nTotal de correspondências: {iptu_geo_coded[~iptu_geo_coded['Padrao'].isna()][col_size].drop_duplicates().shape[0]} / {df_GEO_endereco[col_size].drop_duplicates().shape[0]}\n")

iptu_geo_coded.head()


Total de correspondências: 4911 / 7327



,Empreendimento_Nome,DESC_RUA,NUMERO_RUA,Imovel_residencial,Imovel_vertical,CEP,Idade_predio,Blocos,Unidades_andar,Andares_tipo,...,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,FATOR DE OBSOLESCENCIA,CONTAGEM_UNIDADES,VALOR DO M2 DO TERRENO,AREA DO TERRENO,VALOR DO M2 DE CONSTRUCAO,AREA CONSTRUIDA,Padrao,Fator_obsolescencia_calculado
0,NAO DEFINIDO,ABACATUAJA,500.0,1,0,08190-420,11.0,1.0,8.0,2.0,...,2.0,10.0,0.86,8.0,446.0,500.0,1420.0,466.0,B,0.86
1,PARQUE DOS DOURADOS,ABADIA DOS DOURADOS,166.0,1,1,05000-000,29.0,1.0,4.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.71
2,GOLDEN PALACE,ABAGIBA,540.0,1,1,04294-000,29.0,1.0,4.0,8.0,...,9.0,40.0,0.74,32.0,1557.0,1000.0,2291.0,3424.0,C,0.71
3,RESERVA SAÚDE,ABAGIBA,582.0,1,1,04294-000,8.0,1.0,6.0,17.0,...,18.0,120.0,0.96,102.0,1557.0,2935.0,2291.0,10603.0,C,0.94
4,MISSOURI,ABAGIBA,583.0,1,1,04294-000,29.0,1.0,4.0,19.0,...,20.0,30.0,0.74,76.0,1602.0,1500.0,2291.0,10703.0,C,0.71


In [21]:
# @Checagem de dados (Tipo imóvel, idade e número de andares)
tol_PVTOS = 6
tol_OBSOLENCIA = 0.075

iptu_geo_final = iptu_geo_coded[iptu_geo_coded['Imovel_residencial'] == iptu_geo_coded['Imovel_residencial']]
iptu_geo_final = iptu_geo_final[iptu_geo_final['Imovel_vertical'] == iptu_geo_final['Imovel_vertical']]

# Filter rows where the 'NUMERO_RUA' column from df1 is within the tolerance range of df2
iptu_geo_final = iptu_geo_final[
    (iptu_geo_final['Andares_tipo'] >= iptu_geo_final['QUANTIDADE DE PAVIMENTOS'] - tol_PVTOS) &
    (iptu_geo_final['Andares_tipo'] <= iptu_geo_final['QUANTIDADE DE PAVIMENTOS'] + tol_PVTOS)]

iptu_geo_final = iptu_geo_final[
    (iptu_geo_final['Fator_obsolescencia_calculado'] >= iptu_geo_final['FATOR DE OBSOLESCENCIA'] - tol_OBSOLENCIA) &
    (iptu_geo_final['Fator_obsolescencia_calculado'] <= iptu_geo_final['FATOR DE OBSOLESCENCIA'] + tol_OBSOLENCIA)]

print(
    f"\nOcorrências com alta certeza: {iptu_geo_final[col_size].drop_duplicates().dropna().shape[0]} / {df_GEO_endereco[col_size].drop_duplicates().shape[0]}\n")

cols_final = iptu_geo_final.columns

iptu_geo_final.to_pickle("data_clean/iptu_geo_sabesp.pkl")

iptu_geo_final.head()


Ocorrências com alta certeza: 4421 / 7327



,Empreendimento_Nome,DESC_RUA,NUMERO_RUA,Imovel_residencial,Imovel_vertical,CEP,Idade_predio,Blocos,Unidades_andar,Andares_tipo,...,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,FATOR DE OBSOLESCENCIA,CONTAGEM_UNIDADES,VALOR DO M2 DO TERRENO,AREA DO TERRENO,VALOR DO M2 DE CONSTRUCAO,AREA CONSTRUIDA,Padrao,Fator_obsolescencia_calculado
0,NAO DEFINIDO,ABACATUAJA,500.0,1,0,08190-420,11.0,1.0,8.0,2.0,...,2.0,10.0,0.86,8.0,446.0,500.0,1420.0,466.0,B,0.86
2,GOLDEN PALACE,ABAGIBA,540.0,1,1,04294-000,29.0,1.0,4.0,8.0,...,9.0,40.0,0.74,32.0,1557.0,1000.0,2291.0,3424.0,C,0.71
3,RESERVA SAÚDE,ABAGIBA,582.0,1,1,04294-000,8.0,1.0,6.0,17.0,...,18.0,120.0,0.96,102.0,1557.0,2935.0,2291.0,10603.0,C,0.94
4,MISSOURI,ABAGIBA,583.0,1,1,04294-000,29.0,1.0,4.0,19.0,...,20.0,30.0,0.74,76.0,1602.0,1500.0,2291.0,10703.0,C,0.71
5,VILA PIEMONTE,ABAGIBA,603.0,1,1,04294-000,27.0,1.0,4.0,14.0,...,15.0,20.0,0.80,56.0,1602.0,1000.0,2291.0,6778.0,C,0.74


***
***
***

## Caso relaxado

file_saved = 'data_clean/iptu_geo_sabesp_rlx.pkl'

In [22]:
# @Carregar arquivo salvo
# final_base = pd.read_pickle("data_clean/iptu_geo_sabesp_rlx.pkl")


### **COM** tolerância (SABESP x GeoEmbraesp)

Dados agregados e agrupados
- file_saved = 'data_clean/GEO_endereco.pkl'

In [23]:
# Latitude e longitude já com tolerância ~15m
# merge_GEO_LATLONG = pd.merge(data_GEO_SP,
#                             data_SP_checked[
#                                 ['PDE', 'CODE_ENTIDADE', 'CEP', 'TIPO_RUA', 'DESC_RUA', 'NUMERO_RUA', 'LAT', 'LONG',
#                                  'Status']],
#                             how = 'inner',
#                             on = ['LAT', 'LONG'],
#                             suffixes = ('', '_LATLONG'))

#### **CEP** como agregador e número da rua com tolerância

In [24]:
# tol_NUMERO = 40 # caso relaxado
# tol_NUMERO = 25 # caso relaxado
tol_NUMERO = 12
tol_iptu_NUMERO = 40
tol_iptu_PVTOS = 6
tol_iptu_OBSOLENCIA = 0.075

df_LOGRADOURO, df_end_LOGR = get_cases_tolerance(df1 = data_GEO_SP,
                                                 df2 = data_SP_checked[
                                                     ['PDE', 'CODE_ENTIDADE', 'CEP',
                                                      'TIPO_RUA', 'DESC_RUA', 'NUMERO_RUA',
                                                      'LAT', 'LONG', 'Status']],
                                                 confirmed_cases = df_GEO_merged_cep['PDE'],
                                                 agg_cols = ['CEP'],
                                                 tol_NUMERO = tol_NUMERO,
                                                 column_order = None
                                                 #[col for col in full_cols if col not in ['Bairro', 'Ficha', 'LAT', 'LONG', 'Observacoes', 'data_lancamento']]
                                                 )
df_LOGRADOURO.head()

Merge GEOEMBRAESP - SABESP
Empreendimentos de lados opostos da rua -> eliminados: 16671
Endereços possíveis - IPTU:             2465
Endereços possíveis - SABESP:           4248
Endereços possíveis (novos):            6713


,Empreendimento_Nome,CEP,DESC_RUA,Imovel_residencial,Imovel_vertical,NUMERO_RUA,Idade_predio,Blocos,Unidades_andar,Andares_tipo,...,Vagas_Garagem,M2_util_unidade_tipo,M2_total_unidade_tipo,M2_Terreno,M2_util_cobertura,M2_total_cobertura,RS_total_unidade_tipo_IGPM,RS_por_M2_area_util_IGPM,RS_por_M2_area_total_IGPM,RS_total_cobertura_IGPM
0,VILA DO PARQUE,02144-030,2deg SARGENTO GERALDO BERTI,1,0,20.0,19.0,1.0,4.00,2.0,...,1.0,84.000,105.000,420.000,NaN,NaN,5.126262e+05,6102.709946,4882.167957,-1.000000e+00
1,PARQUE DOS DOURADOS,05586-030,ABADIA DOS DOURADOS,1,1,166.0,29.0,1.0,4.00,8.0,...,2.0,67.150,157.790,1127.310,113.0,223.0,7.679933e+05,11436.936658,4867.157813,1.222726e+06
2,SOUL BROOKLIN,04707-040,ABARA,1,1,47.0,4.0,1.0,13.42,21.0,...,0.0,29.775,54.025,804.285,NaN,NaN,5.741512e+05,19277.443016,10624.505703,-1.000000e+00
3,NAO DEFINIDO,04284-080,ABAUNA,1,0,551.0,11.0,1.0,8.00,3.0,...,2.0,116.000,175.000,810.000,NaN,NaN,1.635443e+06,14098.654854,9345.380742,-1.000000e+00
4,NEW ORLEANS,05725-030,ABDO AMBUBA,1,1,149.0,38.0,1.0,1.00,16.0,...,3.0,244.200,498.000,2200.000,NaN,NaN,-1.000000e+00,-1.000000,-1.000000,-1.000000e+00


In [25]:
# @Merge com GEO + IPTU com tolerancia

possible_tgts_LOGR = merge_df_tolerance(data_cep_iptu = df_end_LOGR,
                                        data_iptu = iptu_agg,
                                        data_imoveis = df_LOGRADOURO,
                                        tol_NUMERO = tol_iptu_NUMERO,
                                        tol_PVTOS = tol_iptu_PVTOS,
                                        tol_OBSOLENCIA = tol_iptu_OBSOLENCIA,
                                        column_order = None)

possible_tgts_LOGR

,DESC_RUA,NUMERO_RUA,Imovel_residencial,Imovel_vertical,QUANTIDADE DE PAVIMENTOS,Andares_tipo,NUMERO_RUA_iptu,TESTADA PARA CALCULO,FATOR DE OBSOLESCENCIA,CONTAGEM_UNIDADES,...,M2_Terreno,M2_util_cobertura,M2_total_cobertura,RS_total_unidade_tipo_IGPM,RS_por_M2_area_util_IGPM,RS_por_M2_area_total_IGPM,RS_total_cobertura_IGPM,CEP,Fator_obsolescencia_calculado,TOL_NUM
26,ABADIA DOS DOURADOS,156.0,1,1,10.0,8.0,182,25.00,0.76,33.0,...,1127.31,113.0,223.0,7.679933e+05,1.143694e+04,4.867158e+03,1.222726e+06,05586-030,0.71,26.0
25,ABADIA DOS DOURADOS,166.0,1,1,10.0,8.0,182,25.00,0.76,33.0,...,1127.31,113.0,223.0,7.679933e+05,1.143694e+04,4.867158e+03,1.222726e+06,05586-030,0.71,16.0
27,ABADIA DOS DOURADOS,176.0,1,1,10.0,8.0,182,25.00,0.76,33.0,...,1127.31,113.0,223.0,7.679933e+05,1.143694e+04,4.867158e+03,1.222726e+06,05586-030,0.71,6.0
56,ABEL TAVARES,999.0,1,1,19.0,19.0,1003,116.00,0.97,456.0,...,5125.03,NaN,NaN,2.926428e+05,6.623882e+03,4.557587e+03,-1.000000e+00,03810-110,0.96,4.0
57,ABEL TAVARES,1001.0,1,1,19.0,19.0,1003,116.00,0.97,456.0,...,5125.03,NaN,NaN,2.926428e+05,6.623882e+03,4.557587e+03,-1.000000e+00,03810-110,0.96,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40263,XAVIER DE ALMEIDA,928.0,1,1,23.0,22.0,918,41.32,0.84,264.0,...,2985.33,NaN,NaN,1.175801e+06,8.380778e+03,4.839191e+03,-1.000000e+00,04211-001,0.81,10.0
40327,ZACARIAS DE GOIS,1288.0,1,1,19.0,15.0,1296,20.00,0.70,30.0,...,821.24,0.0,0.0,8.744481e+12,8.328077e+10,4.265601e+10,0.000000e+00,04610-003,0.66,8.0
40324,ZACARIAS DE GOIS,1296.0,1,1,19.0,15.0,1296,20.00,0.70,30.0,...,821.24,0.0,0.0,8.744481e+12,8.328077e+10,4.265601e+10,0.000000e+00,04610-003,0.66,0.0
40378,ZUMKELLER,792.0,1,1,12.0,11.0,792,30.17,0.74,44.0,...,1665.00,NaN,NaN,1.139446e+06,1.557680e+04,6.502916e+03,-1.000000e+00,02420-001,0.71,0.0


In [26]:
possible_tgts_LOGR_scrap = pd.read_csv(
    'scraps/tol_geo_2810.csv')  # arquivo com merge mais restrito (confiável) <- pode ser alternativa usar todos os casos
# possible_tgts_LOGR_scrap = pd.read_csv('scraps/tol_geo_2810_rlx.csv')
# possible_tgts_LOGR_scrap = pd.read_csv('scraps/tol_geo_2810_num25.csv')

LOGR_scrap_aceitos = verify_targets(df_tgts = possible_tgts_LOGR,
                                    tol_TESTADA = 0.65,
                                    column_order = cols_final,
                                    return_accepted = True,
                                    scrap_tgts = possible_tgts_LOGR_scrap,
                                    filename_scrap = 'scraps/tol_geo.csv',
                                    debugging = False
                                    )

# Exemplo debug
# LOGR_scrap_aceitos = verify_targets(df_tgts = possible_tgts_LOGR,
#                                     tol_TESTADA = 0.65,
#                                     column_order = None, #cols_final,
#                                     return_accepted = True,
#                                     scrap_tgts = possible_tgts_LOGR_scrap,
#                                     filename_scrap = 'scraps/tol_geo.csv',
#                                     debugging = True # se True exibe coluna num_iptu
#                                     )

# Exemplo testada mais restritiva
# LOGR_scrap_aceitos = verify_targets(df_tgts = possible_tgts_LOGR,
#                                    tol_TESTADA = 0.80,
#                                    column_order = None,
#                                    return_accepted = False,
#                                    scrap_tgts = possible_tgts_scrap,
#                                    filename_scrap = 'scraps/tol_geo.csv',
#                                    )

LOGR_scrap_aceitos

Aceitos: 26
Rejeitados: 96


,Empreendimento_Nome,DESC_RUA,NUMERO_RUA,Imovel_residencial,Imovel_vertical,CEP,Idade_predio,Blocos,Unidades_andar,Andares_tipo,...,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,FATOR DE OBSOLESCENCIA,CONTAGEM_UNIDADES,VALOR DO M2 DO TERRENO,AREA DO TERRENO,VALOR DO M2 DE CONSTRUCAO,AREA CONSTRUIDA,Padrao,Fator_obsolescencia_calculado
44,WORKSTATION BELÉM,ALVARO RAMOS,860.0,0,1,03330-002,6.0,1.00,6.00,10.00,...,11.0,46.45,0.99,0.0,2517.0,915.000000,2763.0,3644.0,C,0.96
28,ALTOS DE SANTANA,ANA DE BARROS,296.0,1,1,02423-020,33.0,1.00,4.00,8.00,...,9.0,24.00,0.71,64.0,1769.0,1350.000000,2291.0,8404.0,C,0.66
115,THE PENÍNSULA RESIDENCE,BARAO DE CASTRO LIMA,22.0,1,1,05685-040,29.0,1.00,1.00,7.00,...,10.0,30.00,0.74,28.0,1657.0,2181.000000,3348.0,8792.0,D,0.71
113,REFERENCE BUTANTÃ,CEL. FERREIRA LEAL,159.0,1,1,05589-090,10.0,1.00,6.00,10.00,...,11.0,22.32,0.94,60.0,2195.0,1265.000000,2291.0,7123.0,C,0.92
96,DIANA,CROATA,397.0,1,1,05056-020,39.0,1.00,4.00,13.00,...,15.0,18.34,0.57,52.0,2702.0,1001.461538,3279.0,6552.0,C,0.57
59,DON KLABIN,DIONISIO DA COSTA,370.0,1,1,04117-110,11.0,1.00,2.00,18.00,...,19.0,47.96,0.94,36.0,4538.0,2071.000000,3348.0,11685.0,D,0.91
15,SAINT PAUL GARDEN,GILBERTO SAMPAIO,17.0,1,1,02043-020,32.0,1.00,3.00,8.00,...,10.0,22.60,0.70,24.0,2579.0,1344.000000,3348.0,6676.0,D,0.67
110,YBIRÁ,GIRASSOL,986.0,1,1,05433-002,6.0,1.00,6.00,18.00,...,19.0,36.05,0.97,110.0,6843.0,1675.000000,3279.0,13112.0,C,0.96
35,MONTEREY,HENRIQUE SERTORIO,308.0,1,1,03066-065,24.0,1.00,4.00,17.00,...,18.0,19.90,0.79,68.0,2687.0,1138.000000,3279.0,8518.0,C,0.78
7,MIAMI OFFICE,IGUATEMI,232.0,0,1,01451-010,36.0,1.00,8.00,12.00,...,13.0,15.32,0.63,0.0,11777.0,895.000000,3103.0,6864.0,C,0.62


In [23]:
# SE QUISER CONFERIR IMPACTO DO MERGE NOS PDES ENCONTRADOS
# data_SP_checked[(aux_scrap_aceitos['DESC_RUA'] == 'ALVARO RAMOS') &
#                 ((data_SP_checked['NUMERO_RUA'] == 860)
#                 | (data_SP_checked['NUMERO_RUA'] == 896))
# ]
# qwe = pd.merge(aux_scrap_aceitos[['DESC_RUA', 'NUMERO_RUA', 'NUMERO_RUA_iptu']], data_SP_checked[['DESC_RUA', 'NUMERO_RUA', 'MUNICIPIO']],
#                how = 'left', on = ['DESC_RUA', 'NUMERO_RUA']).drop_duplicates().dropna()
# 
# qwe = pd.merge(aux_scrap_aceitos[['DESC_RUA','NUMERO_RUA_iptu']].rename(columns={'NUMERO_RUA_iptu' : 'NUMERO_RUA'}), data_SP_checked[['DESC_RUA', 'NUMERO_RUA', 'MUNICIPIO']],
#                how = 'left', on = ['DESC_RUA', 'NUMERO_RUA']).drop_duplicates().dropna()

In [27]:
df_RUA, df_end_RUA = get_cases_tolerance(df1 = data_GEO_SP,
                                         df2 = data_SP_checked[
                                             ['PDE', 'CODE_ENTIDADE', 'CEP',
                                              'TIPO_RUA', 'DESC_RUA', 'NUMERO_RUA',
                                              'LAT', 'LONG', 'Status']],
                                         confirmed_cases = df_GEO_merged_cep['PDE'],
                                         agg_cols = ['DESC_RUA'],
                                         tol_NUMERO = tol_NUMERO,
                                         column_order = None
                                         #[col for col in full_cols if col not in ['Bairro', 'Ficha', 'LAT', 'LONG', 'Observacoes', 'data_lancamento']]
                                         )
df_RUA.head()

Merge GEOEMBRAESP - SABESP
Empreendimentos de lados opostos da rua -> eliminados: 32305
Endereços possíveis - IPTU:             2428
Endereços possíveis - SABESP:           4330
Endereços possíveis (novos):            6758


,Empreendimento_Nome,CEP,DESC_RUA,Imovel_residencial,Imovel_vertical,NUMERO_RUA,Idade_predio,Blocos,Unidades_andar,Andares_tipo,...,Vagas_Garagem,M2_util_unidade_tipo,M2_total_unidade_tipo,M2_Terreno,M2_util_cobertura,M2_total_cobertura,RS_total_unidade_tipo_IGPM,RS_por_M2_area_util_IGPM,RS_por_M2_area_total_IGPM,RS_total_cobertura_IGPM
0,PARQUE DOS DOURADOS,05586-030,ABADIA DOS DOURADOS,1,1,166.0,29.0,1.0,4.00,8.0,...,2.0,67.150,157.790,1127.310,113.0,223.0,7.679933e+05,11436.936658,4867.157813,1.222726e+06
1,SOUL BROOKLIN,04707-040,ABARA,1,1,47.0,4.0,1.0,13.42,21.0,...,0.0,29.775,54.025,804.285,NaN,NaN,5.741512e+05,19277.443016,10624.505703,-1.000000e+00
2,NAO DEFINIDO,04284-080,ABAUNA,1,0,551.0,11.0,1.0,8.00,3.0,...,2.0,116.000,175.000,810.000,NaN,NaN,1.635443e+06,14098.654854,9345.380742,-1.000000e+00
3,NEW ORLEANS,05725-030,ABDO AMBUBA,1,1,149.0,38.0,1.0,1.00,16.0,...,3.0,244.200,498.000,2200.000,NaN,NaN,-1.000000e+00,-1.000000,-1.000000,-1.000000e+00
4,NEWCASTLE,05725-030,ABDO AMBUBA,1,1,249.0,39.0,1.0,1.00,16.0,...,3.0,227.110,474.180,1912.000,NaN,NaN,-1.000000e+00,-1.000000,-1.000000,-1.000000e+00


In [28]:
# @Merge com GEO + IPTU com tolerancia

possible_tgts_RUA = merge_df_tolerance(data_cep_iptu = df_end_RUA,
                                       data_iptu = iptu_agg,
                                       data_imoveis = df_RUA,
                                       tol_NUMERO = tol_iptu_NUMERO,
                                       tol_PVTOS = tol_iptu_PVTOS,
                                       tol_OBSOLENCIA = tol_iptu_OBSOLENCIA,
                                       column_order = None)

possible_tgts_RUA

,DESC_RUA,NUMERO_RUA,Imovel_residencial,Imovel_vertical,QUANTIDADE DE PAVIMENTOS,Andares_tipo,NUMERO_RUA_iptu,TESTADA PARA CALCULO,FATOR DE OBSOLESCENCIA,CONTAGEM_UNIDADES,...,M2_Terreno,M2_util_cobertura,M2_total_cobertura,RS_total_unidade_tipo_IGPM,RS_por_M2_area_util_IGPM,RS_por_M2_area_total_IGPM,RS_total_cobertura_IGPM,CEP,Fator_obsolescencia_calculado,TOL_NUM
1,ABADIA DOS DOURADOS,156.0,1,1,10.0,8.0,182,25.00,0.76,33.0,...,1127.31,113.0,223.0,7.679933e+05,1.143694e+04,4.867158e+03,1.222726e+06,05586-030,0.71,26.0
0,ABADIA DOS DOURADOS,166.0,1,1,10.0,8.0,182,25.00,0.76,33.0,...,1127.31,113.0,223.0,7.679933e+05,1.143694e+04,4.867158e+03,1.222726e+06,05586-030,0.71,16.0
30,ABEL TAVARES,999.0,1,1,19.0,19.0,1003,116.00,0.97,456.0,...,5125.03,NaN,NaN,2.926428e+05,6.623882e+03,4.557587e+03,-1.000000e+00,03810-110,0.96,4.0
31,ABEL TAVARES,1001.0,1,1,19.0,19.0,1003,116.00,0.97,456.0,...,5125.03,NaN,NaN,2.926428e+05,6.623882e+03,4.557587e+03,-1.000000e+00,03810-110,0.96,2.0
27,ABEL TAVARES,1003.0,1,1,19.0,19.0,1003,116.00,0.97,456.0,...,5125.03,NaN,NaN,2.926428e+05,6.623882e+03,4.557587e+03,-1.000000e+00,03810-110,0.96,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38820,XAVIER DE ALMEIDA,928.0,1,1,23.0,22.0,918,41.32,0.84,264.0,...,2985.33,NaN,NaN,1.175801e+06,8.380778e+03,4.839191e+03,-1.000000e+00,04211-001,0.81,10.0
38824,ZACARIAS DE GOIS,1288.0,1,1,19.0,15.0,1296,20.00,0.70,30.0,...,821.24,0.0,0.0,8.744481e+12,8.328077e+10,4.265601e+10,0.000000e+00,04610-003,0.66,8.0
38821,ZACARIAS DE GOIS,1296.0,1,1,19.0,15.0,1296,20.00,0.70,30.0,...,821.24,0.0,0.0,8.744481e+12,8.328077e+10,4.265601e+10,0.000000e+00,04610-003,0.66,0.0
38875,ZUMKELLER,792.0,1,1,12.0,11.0,792,30.17,0.74,44.0,...,1665.00,NaN,NaN,1.139446e+06,1.557680e+04,6.502916e+03,-1.000000e+00,02420-001,0.71,0.0


In [29]:
# possible_tgts_RUA_scrap = pd.read_csv('scraps/tol_geo_rua_2810_num25.csv')
possible_tgts_RUA_scrap = pd.read_csv('scraps/tol_geo_rua_2810.csv')

RUA_scrap_aceitos = verify_targets(df_tgts = possible_tgts_RUA,
                                   tol_TESTADA = 0.65,
                                   column_order = cols_final,
                                   return_accepted = True,
                                   scrap_tgts = possible_tgts_RUA_scrap,
                                   filename_scrap = 'scraps/tol_geo_rua_2810_num25.csv',
                                   debugging = False
                                   )

# Executa scrapping
# RUA_scrap_aceitos = verify_targets(df_tgts = possible_tgts_RUA,
#                                    tol_TESTADA = 0.65,
#                                    column_order = cols_final,
#                                    return_accepted = True,
#                                    scrap_tgts = None,
#                                    filename_scrap = 'scraps/tol_geo_rua_2810_num25.csv',
#                                    debugging = False
#                                    )


RUA_scrap_aceitos

Aceitos: 23
Rejeitados: 87


,Empreendimento_Nome,DESC_RUA,NUMERO_RUA,Imovel_residencial,Imovel_vertical,CEP,Idade_predio,Blocos,Unidades_andar,Andares_tipo,...,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,FATOR DE OBSOLESCENCIA,CONTAGEM_UNIDADES,VALOR DO M2 DO TERRENO,AREA DO TERRENO,VALOR DO M2 DE CONSTRUCAO,AREA CONSTRUIDA,Padrao,Fator_obsolescencia_calculado
26,ALTOS DE SANTANA,ANA DE BARROS,296.0,1,1,02423-020,33.0,1.00,4.00,8.00,...,9.0,24.00,0.71,64.0,1769.0,1350.000000,2291.0,8404.0,C,0.66
105,THE PENÍNSULA RESIDENCE,BARAO DE CASTRO LIMA,22.0,1,1,05685-040,29.0,1.00,1.00,7.00,...,10.0,30.00,0.74,28.0,1657.0,2181.000000,3348.0,8792.0,D,0.71
90,DIANA,CROATA,397.0,1,1,05056-020,39.0,1.00,4.00,13.00,...,15.0,18.34,0.57,52.0,2702.0,1001.461538,3279.0,6552.0,C,0.57
54,DON KLABIN,DIONISIO DA COSTA,370.0,1,1,04117-110,11.0,1.00,2.00,18.00,...,19.0,47.96,0.94,36.0,4538.0,2071.000000,3348.0,11685.0,D,0.91
14,SAINT PAUL GARDEN,GILBERTO SAMPAIO,17.0,1,1,02043-020,32.0,1.00,3.00,8.00,...,10.0,22.60,0.70,24.0,2579.0,1344.000000,3348.0,6676.0,D,0.67
102,YBIRÁ,GIRASSOL,986.0,1,1,05433-002,6.0,1.00,6.00,18.00,...,19.0,36.05,0.97,110.0,6843.0,1675.000000,3279.0,13112.0,C,0.96
8,MIAMI OFFICE,IGUATEMI,232.0,0,1,01451-010,36.0,1.00,8.00,12.00,...,13.0,15.32,0.63,0.0,11777.0,895.000000,3103.0,6864.0,C,0.62
103,SUNSET PLACE,ISABEL DE CASTELA,424.0,1,1,05445-010,24.0,1.00,1.00,17.00,...,21.0,24.00,0.80,18.0,8377.0,1020.000000,3488.0,7908.0,E,0.78
73,TALENT OFFICE CENTER,JOAO CACHOEIRA,474.0,0,1,04535-001,29.0,1.00,10.00,10.00,...,11.0,20.00,0.74,0.0,9858.0,1000.000000,3103.0,7170.0,C,0.71
74,UBER ITAIM,JOAO CACHOEIRA,1549.0,1,1,04535-015,10.0,1.00,9.00,13.00,...,16.0,30.00,0.94,122.0,9456.0,1200.000000,3348.0,11018.0,D,0.92


### Resultados

In [30]:
tolerance_cases = pd.concat([LOGR_scrap_aceitos, RUA_scrap_aceitos], axis = 0).drop_duplicates()

# print(f"Novos consumidores: [{tolerance_cases['Total_Unidades'].sum():.0f}, {tolerance_cases['CONTAGEM_UNIDADES'].sum():.0f}]")
final_base = pd.concat([iptu_geo_final, tolerance_cases], axis = 0).drop_duplicates()

print(f"Total consumidores: [{final_base['Total_Unidades'].sum():.0f}, {final_base['CONTAGEM_UNIDADES'].sum():.0f}]")

final_base.to_pickle("data_clean/iptu_geo_sabesp_rlx.pkl")

final_base

Total consumidores: [339349, 287270]


,Empreendimento_Nome,DESC_RUA,NUMERO_RUA,Imovel_residencial,Imovel_vertical,CEP,Idade_predio,Blocos,Unidades_andar,Andares_tipo,...,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,FATOR DE OBSOLESCENCIA,CONTAGEM_UNIDADES,VALOR DO M2 DO TERRENO,AREA DO TERRENO,VALOR DO M2 DE CONSTRUCAO,AREA CONSTRUIDA,Padrao,Fator_obsolescencia_calculado
0,NAO DEFINIDO,ABACATUAJA,500.0,1,0,08190-420,11.0,1.00,8.00,2.0,...,2.0,10.00,0.86,8.0,446.0,500.0,1420.0,466.0,B,0.86
2,GOLDEN PALACE,ABAGIBA,540.0,1,1,04294-000,29.0,1.00,4.00,8.0,...,9.0,40.00,0.74,32.0,1557.0,1000.0,2291.0,3424.0,C,0.71
3,RESERVA SAÚDE,ABAGIBA,582.0,1,1,04294-000,8.0,1.00,6.00,17.0,...,18.0,120.00,0.96,102.0,1557.0,2935.0,2291.0,10603.0,C,0.94
4,MISSOURI,ABAGIBA,583.0,1,1,04294-000,29.0,1.00,4.00,19.0,...,20.0,30.00,0.74,76.0,1602.0,1500.0,2291.0,10703.0,C,0.71
5,VILA PIEMONTE,ABAGIBA,603.0,1,1,04294-000,27.0,1.00,4.00,14.0,...,15.0,20.00,0.80,56.0,1602.0,1000.0,2291.0,6778.0,C,0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,O ATENEU,RAUL POMPEIA,175.0,1,1,05025-010,30.0,1.00,4.00,19.0,...,21.0,28.30,0.73,78.0,5272.0,1675.0,3279.0,11622.0,C,0.70
21,RIO BRANCO,RUBIACEA,137.0,1,1,02335-020,19.0,1.01,4.00,8.0,...,9.0,22.10,0.86,32.0,2094.0,1372.0,2291.0,5208.0,C,0.83
123,PLANO & VILA CARMOSINA,VICTORIO SANTIM,2798.0,1,1,08290-001,5.0,2.00,11.76,21.0,...,21.0,30.00,0.98,494.0,808.0,4351.0,1807.0,20418.0,C,0.96
41,PORTAL VILA PRUDENTE - VILA ANHAIA MELLO,VILA EMA,2142.0,1,1,03282-000,4.0,1.01,12.62,21.0,...,22.0,51.69,0.99,470.0,1541.0,3045.0,2291.0,19740.0,C,0.97
